In [1]:
%load_ext autoreload
%autoreload 2

from src.data.mnist import MNISTDataModule
from src.models.mnist import 

from src.inference import MonteCarloInference
from src.samplers import Hamiltonian, MetropolisHastings

import torch
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dm = MNISTDataModule(32)
dm.setup()

In [4]:
train_data, test_data = dm.mnist_train, dm.mnist_test

In [5]:
X_train, y_train = train_data[:]
X_test, y_test = test_data[:]

In [6]:
class MNISTModel(Net):

    def forward(self, x):
        x = x.flatten(-2, -1)
        return self.ffnn(x)
    
    def log_likelihood(self, x: torch.FloatTensor, y: torch.LongTensor):
        """Returns log p(y | x, theta)"""
        logits = self.forward(x)
        observation_model = torch.distributions.Categorical(logits=logits)
        return observation_model.log_prob(y).sum()


In [25]:
model = MNISTModel(784, 10, [100])

In [29]:
model.parameters()

<generator object Module.parameters at 0x14bfce890>

In [27]:
sampler = Hamiltonian(step_size=0.1, n_steps=1)
inference = MonteCarloInference(sampler=sampler, burn_in=0, n_samples=100)
inference.fit(model, train_data)

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
sample_df = pd.DataFrame({f"$c_{i}$" : c for i, c in enumerate(inference.samples_.T[100:105])})
plot_data = sample_df.reset_index().melt(id_vars="index")
fig = plt.figure()
grid_spec = fig.add_gridspec(2, 1, height_ratios=(2, 7))

ax_line = fig.add_subplot(grid_spec[1, 0])
ax_marg = fig.add_subplot(grid_spec[0, 0], sharex=ax_line)

sns.lineplot(x = "value", y="index", hue="variable", ax=ax_line, data=plot_data, sort=False, legend=False)
sns.kdeplot(x = "value", hue="variable", data=plot_data, legend=False)

plt.show()

In [ ]:
y_pred_samples = inference.predictive(X_test)
y_pred = torch.tensor([v.bincount(minlength=10).argmax() for v in y_pred_samples.argmax(-1)])

In [ ]:
y_pred_samples

In [ ]:
y_test

In [ ]:
pca = PCA(2).fit(X_train.flatten(-2, -1))

In [ ]:
certainty = (y_pred == y_pred_samples.argmax(-1)).sum(0) / 10

In [ ]:
u, v = pca.transform(X_test.flatten(-2, -1)).T
# sns.scatterplot(x=u, y=v, hue=y_pred)
sns.scatterplot(x=u, y=v, hue=certainty)
plt.figure()
sns.scatterplot(x=u, y=v, hue=y_pred)

In [ ]:
y_pred_samples